In [21]:
import hyperopt as hp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import pathlib 
import os

from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from hyperopt import Trials, hp, fmin, tpe, STATUS_OK, space_eval 



In [22]:
cur_path  = os.getcwd()

df = pd.read_csv(cur_path + '/processed.csv')
pd.set_option('display.min_rows', 150)


In [23]:
df['label'] = df['label'].eq('Churn').mul(1)

X = df.drop("label", axis = 1)
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#################################################
space = {
    'n_estimators': hp.choice('n_estimators', range(50, 500)),
    'max_depth': hp.choice('max_depth', [1, 5, 10, 20, 50, 75, 100, 150, 200]),
    'min_samples_split': hp.choice('min_samples_split', [2, 3, 4, 5, 10, 20]),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 3, 4, 5]),
    'bootstrap': hp.choice('bootstrap', [True, False]),
    'criterion': hp.choice('criterion', ['gini', 'entropy']),
    'max_features': hp.choice('max_features', ['sqrt', 'log2', None])
}

kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)

def objective(params):
    params['max_depth'] = int(params['max_depth'])
    clf = RandomForestClassifier(**params)
    print(params)
    scores = cross_val_score(clf, X_train, y_train, cv = kfold, scoring = 'f1', n_jobs = -1)
    print(scores)
    best_score = mean(scores) 
    loss = -best_score
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

num_trials = Trials()
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 1, trials = num_trials)

randomForest = RandomForestClassifier(n_estimators = space_eval(space, best)['n_estimators'], max_depth = space_eval(space, best)['max_depth'],
                                     min_samples_split = space_eval(space, best)['min_samples_split'], min_samples_leaf = space_eval(space, best)['min_samples_leaf'],
                                     bootstrap = space_eval(space, best)['bootstrap'], criterion = space_eval(space, best)['criterion'], max_features = space_eval(space, best)['max_features'])
randomForest.fit(X_train, y_train)


######################################################################

model = RandomForestClassifier()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)


ERROR in rec_eval                                    
EXCEPTION                                            
<class 'ValueError'>                                 
high <= 0                                            
NODE                                                 
0 randint                                            
1   Literal{0}
2  size =
3   len
4     array_union
5       array_union
6         array_union
7           Literal{new_ids}
8  rng =
9   Literal{rng-placeholder}
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]


ValueError: high <= 0

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
f1_score = f1_score(y_test, y_pred)

print('Accuracy', accuracy)
print('F1 Score', f1_score)
print('Classification Report:\n', classification_report(y_test, y_pred))